### <p style="font-family: Arial; color: gold; font-weight: bold;">**create by Tom Tan in 8.30.2024** </p>
##### Now one notebook will deal with all the prefix, the downside is that the define properties will be the same for all the prefix.

***
# **1. Imports**

In [1]:
import os, re, glob
import pandas as pd

import get_properties_functions_for_WI as gp

common_structure_folder = "1.common_structure"
log_files_folder = "2.log_files"
sdf_files_folder = "3.sdf_files"
temp_folder = "temp"
atom_mappings_folder = "4.atom_mappings"
output_folder = "5.postprocessed_results"

***
# **2. Import the atom map from preprocess notebook**
### <p style="font-family: Arial; color: gold; font-weight: bold;"> **Grep all available prefix based on the log file** </p>

In [2]:
prefixs = {}
for file in glob.glob("*.xlsx", root_dir=atom_mappings_folder):
    key = re.search(r"^(\D+)_atom_map", file)
    if key and key.group(1) in prefixs:
        prefixs[key.group(1)].append(file)
    else:
        prefixs[key.group(1)] = [file]

In [3]:
atom_map_df_all = {}
for prefix in prefixs:
    atom_mappings = atom_mappings_folder + os.sep + prefix + "_atom_map.xlsx"
    atom_map_df = pd.read_excel(
        atom_mappings, "Sheet1", index_col=0, header=0, engine="openpyxl"
    )
    # add the log_files_folder before all the log_names cells
    atom_map_df["log_name"] = log_files_folder + os.sep + atom_map_df["log_name"]
    atom_map_df_all[prefix] = atom_map_df.copy(deep=True)
    print(f"prefix: {prefix}")
    display(atom_map_df.head(3))

prefix: pyrdz


,log_name,N3,N4,C5,C6,C7,C2,C1,H1
0,2.log_files\pyrdz1_conf-1_openshell,N7,N6,C5,C4,C3,C2,C1,H8
1,2.log_files\pyrdz2_conf-1_openshell,C3,N4,N5,C6,C7,C2,C1,H8
2,2.log_files\pyrdz3_conf-1_openshell,N8,N7,C6,C5,C4,C3,C2,H12


prefix: pyrd


,log_name,C3,C4,N5,C6,C7,C2,C1,H1
0,2.log_files\pyrd1_conf-1_openshell,C5,C4,N3,C11,C10,C2,C1,H12
1,2.log_files\pyrd2_conf-1_openshell,C11,N10,C9,C4,C3,C2,C1,H12
2,2.log_files\pyrd3_conf-1_openshell,C3,C4,N5,C6,C7,C2,C1,H12


prefix: pyrmd


,log_name,N3,C4,C5,C6,N7,C2,C1,H1
0,2.log_files\pyrmd1_conf-1_openshell,N3,C4,C5,C10,N11,C2,C1,H12
1,2.log_files\pyrmd2_conf-1_openshell_resubmit,C10,N9,C8,C3,N11,C2,C1,H12
2,2.log_files\pyrmd3_conf-2_openshell,N12,C11,C10,C5,N4,C3,C2,H16


prefix: pyrz


,log_name,C3,N4,C5,C6,N7,C2,C1,H1
0,2.log_files\pyrz1_conf-1_openshell,C11,N10,C5,C4,N3,C2,C1,H12
1,2.log_files\pyrz2_conf-1_openshell,C12,N11,C10,C5,N4,C3,C2,H16
2,2.log_files\pyrz3_conf-1_openshell,C7,N6,C5,C4,N3,C2,C1,H8


# **3. Define Properties to Collect**
### <p style="font-family: Arial; color: gold"> !!!User input required, Change/comment the properties block to the one you want to collect. </p>

In [4]:
for prefix, _ in atom_map_df_all.items():
    print(f"processing prefix: {prefix}")
    pd.set_option("display.max_columns", None)
    # ---------------GoodVibes Engergies---------------
    # uses the GoodVibes 2021 Branch (Jupyter Notebook Compatible)
    # calculates the quasi harmonic corrected G(T) and single point corrected G(T) as well as other thermodynamic properties
    # inputs: dataframe, temperature
    df = atom_map_df_all[prefix].copy(deep=True)
    df = gp.get_goodvibes_e(df, 298.15)

    # ---------------Frontier Orbitals-----------------
    # E(HOMO), E(LUMO), mu(chemical potential or negative of molecular electronegativity), eta(hardness/softness), omega(electrophilicity index)
    df = gp.get_frontierorbs(df)

    # ---------------Polarizability--------------------
    # Exact polarizability
    df = gp.get_polarizability(df)

    # ---------------Dipole----------------------------
    # Total dipole moment magnitude in Debye
    df = gp.get_dipole(df)

    # ---------------Volume----------------------------
    # Molar volume
    # requires the Gaussian keyword = "volume" in the .com file
    df = gp.get_volume(df)

    # ---------------SASA------------------------------
    # Uses morfeus to calculat sovlent accessible surface area and the volume under the SASA
    df = gp.get_SASA(df)

    # ---------------NBO-------------------------------
    # natural charge from NBO
    # requires the Gaussian keyword = "pop=nbo7" in the .com file
    nbo_list = ["C1", "C2"]
    df = gp.get_nbo(df, nbo_list)

    # ---------------NMR-------------------------------
    # isotropic NMR shift
    # requires the Gaussian keyword = "nmr=giao" in the .com file
    # nmr_list = ["C1", "C2"]
    # df = gp.get_nmr(df, nmr_list)

    # ---------------Distance--------------------------
    # distance between 2 atoms
    dist_list_of_lists = [["C1", "C2"]]
    df = gp.get_distance(df, dist_list_of_lists)

    # ---------------Angle-----------------------------
    # angle between 3 atoms
    # angle_list_of_lists = [["C5", "N1", "C1"]]
    # df = gp.get_angles(df, angle_list_of_lists)

    # ---------------Dihedral--------------------------
    # dihedral angle between 4 atoms
    # dihedral_list_of_lists = [["C4", "C5", "N1", "C1"], ["C2", "C1", "N1", "C5"]]
    # df = gp.get_dihedral(df, dihedral_list_of_lists)

    # ---------------Vbur Scan-------------------------
    # uses morfeus to calculate the buried volume at a series of radii (including hydrogens)
    # inputs: dataframe, list of atoms, start_radius, end_radius, and step_size
    # if you only want a single radius, put the same value for start_radius and end_radius (keep step_size > 0)
    vbur_list = ["C1", "C2"]
    df = gp.get_vbur_scan(df, vbur_list, 2, 2, 0.5)

    # ---------------Sterimol morfeus------------------
    # uses morfeus to calculate Sterimol L, B1, and B5 values
    # NOTE: this is much faster than the corresponding DBSTEP function (recommendation: use as default/if you don't need Sterimol2Vec)
    sterimol_list_of_lists = [["C1", "C2"]]
    df = gp.get_sterimol_morfeus(df, sterimol_list_of_lists)

    # ---------------Buried Sterimol-------------------
    # uses morfeus to calculate Sterimol L, B1, and B5 values within a given sphere of radius r_buried
    # atoms outside the sphere + 0.5 vdW radius are deleted and the Sterimol vectors are calculated
    # for more information: https://kjelljorner.github.io/morfeus/sterimol.html
    # inputs: dataframe, list of atom pairs, r_buried
    # sterimol_list_of_lists = [["C1", "C2"]]
    # df = gp.get_buried_sterimol(df, sterimol_list_of_lists, 5.5)

    # ---------------Sterimol DBSTEP-------------------
    # uses DBSTEP to calculate Sterimol L, B1, and B5 values
    # default grid point spacing (0.05 Angstrom) is used (can use custom spacing or vdw radii in the get_properties_functions script)
    # more info here: https://github.com/patonlab/DBSTEP
    # NOTE: this takes longer than the morfeus function (recommendation: only use this if you need Sterimol2Vec)
    # sterimol_list_of_lists = [["N1", "C1"], ["N1", "C5"]]
    # df = gp.get_sterimol_dbstep(df, sterimol_list_of_lists)

    # ---------------Sterimol2Vec----------------------
    # uses DBSTEP to calculate Sterimol Bmin and Bmax values at intervals from 0 to end_radius, with a given step_size
    # default grid point spacing (0.05 Angstrom) is used (can use custom spacing or vdw radii in the get_properties_functions script)
    # more info here: https://github.com/patonlab/DBSTEP
    # inputs: dataframe, list of atom pairs, end_radius, and step_size
    # sterimol2vec_list_of_lists = [["N1", "C1"], ["N1", "C5"]]
    # df = gp.get_sterimol2vec(df, sterimol2vec_list_of_lists, 1, 1.0)

    # ---------------Pyramidalization------------------
    # uses morfeus to calculate pyramidalization based on the 3 atoms in closest proximity to the defined atom
    # collects values based on two definitions of pyramidalization
    # details on these values can be found here: https://kjelljorner.github.io/morfeus/pyramidalization.html
    # pyr_list = ["C1"]
    # df = gp.get_pyramidalization(df, pyr_list)

    # ---------------Plane Angle-----------------------
    # !plane angle between 2 planes (each defined by 6 atoms)
    # planeangle_list_of_lists = [["N1", "C1", "C5"], ["C2", "C3", "C4"]]
    # df = gp.get_planeangle(df, planeangle_list_of_lists)

    # --------------LP energy - custom from first cell---------------
    # lp_list = ["N1"]
    # df = gp.get_one_lp_energy(df, lp_list)

    # ---------------Time----------------------------------
    # returns the total CPU time and total Wall time (not per subjob) because we are pioneers
    # if used in summary df, will give the average (not Boltzmann average) in the Boltzmann average column
    # df = gp.get_time(df)

    # ---------------ChelpG----------------------------
    # ChelpG ESP charge
    # requires the Gaussian keyword = "pop=chelpg" in the .com file
    # a_list = ["C1", "C2", "C3", "C4", "C5", "N1"]
    # df = gp.get_chelpg(df, a_list)

    # ---------------Hirshfeld-------------------------
    # Hirshfeld charge, CM5 charge, Hirshfeld atom dipole
    # requires the Gaussian keyword = "pop=hirshfeld" in the .com file
    # a_list = ["C1", "C2", "C3", "C4", "C5", "N1"]
    # df = gp.get_hirshfeld(df, a_list)

    # !new functions below!
    # ---------------Natural Bond Order (total/covalent/ionic)-------------------------
    # Natural Bond Order (total/covalent/ionic) between 2 atoms, might return non-numerical values
    # requires Natural Resonance Theory Analysis in Gaussian input file ("$nbo nrt $end" in the .com file)
    natural_bond_order_list = [["C1", "C2"]]
    df = gp.get_natural_bond_order(df, natural_bond_order_list)

    # ---------------Natural Atomic Valencies, Electron Counts, and Charges-------------------------
    # Natural Atomic Valencies, Electron Counts, and Charges of a atom
    # requires Natural Resonance Theory Analysis in Gaussian input file ("$nbo nrt $end" in the .com file)
    natural_atomic_valencies_list = ["C1", "C2"]
    df = gp.get_natural_atomic_valencies(df, natural_atomic_valencies_list)

    display(df)
    # copy the changes back to atom_map_df_all
    atom_map_df_all[prefix] = df.copy(deep=True)
    
# delete the "Goodvibes_output.dat" temp file
if os.path.exists("Goodvibes_output.dat"):
    os.remove("Goodvibes_output.dat")

processing prefix: pyrdz
Goodvibes function has completed
Frontier orbitals function has completed
Polarizability function has completed
Dipole function has completed
Volume function has completed
SASA function has completed
NBO function has completed for ['C1', 'C2']
Distance function has completed for [['C1', 'C2']]
Vbur scan function has completed for ['C1', 'C2'] from 2 to 2
Morfeus Sterimol function has completed for [['C1', 'C2']]
****No Natural Bond Order section found in: 2.log_files\pyrdz1_conf-1_openshell.log
****No Natural Bond Order section found in: 2.log_files\pyrdz2_conf-1_openshell.log
****No Natural Bond Order section found in: 2.log_files\pyrdz3_conf-1_openshell.log
Natural Bond Order function has completed for [['C1', 'C2']]
****No Natural Atomic Valencies section found in: 2.log_files\pyrdz1_conf-1_openshell.log
****No Natural Atomic Valencies section found in: 2.log_files\pyrdz2_conf-1_openshell.log
****No Natural Atomic Valencies section found in: 2.log_files\pyrd

,log_name,N3,N4,C5,C6,C7,C2,C1,H1,E_spc (Hartree),ZPE(Hartree),H_spc(Hartree),T*S,T*qh_S,G(T)_spc(Hartree),qh_G(T)_spc(Hartree),T,HOMO,LUMO,μ,η,ω,polar_iso(Debye),polar_aniso(Debye),dipole(Debye),volume(Bohr_radius³/mol),SASA_surface_area(Å²),SASA_volume(Å³),SASA_sphericity,NBO_charge_C1,NBO_charge_C2,distance_C1_C2(Å),%Vbur_C1_2.0Å,%Vbur_C2_2.0Å,Sterimol_L_C1_C2(Å)_morfeus,Sterimol_B1_C1_C2(Å)_morfeus,Sterimol_B5_C1_C2(Å)_morfeus,C1_C2_Bond_Order,Natural_Valency_C1,Natural_Valency_C2
0,2.log_files\pyrdz1_conf-1_openshell,N7,N6,C5,C4,C3,C2,C1,H8,-302.930027,0.090330,-302.833243,0.036086,0.036089,-302.869329,-302.869332,298.15,-0.26296,-0.02244,-0.14270,0.24052,0.04233,71.9025,55.6433,4.0685,794.393,246.781337,330.850635,0.937403,-0.25250,0.02928,1.40721,84.743027,95.896823,6.652549,1.700000,3.261975,no data,no data,no data
1,2.log_files\pyrdz2_conf-1_openshell,C3,N4,N5,C6,C7,C2,C1,H8,-302.930293,0.090405,-302.833411,0.036145,0.036148,-302.869556,-302.869559,298.15,-0.27376,-0.01060,-0.14218,0.26316,0.03841,71.7614,53.4032,4.6724,1032.521,245.501238,329.913006,0.940510,-0.24648,-0.15464,1.40277,84.894757,96.923425,6.157663,1.700202,3.270340,no data,no data,no data
2,2.log_files\pyrdz3_conf-1_openshell,N8,N7,C6,C5,C4,C3,C2,H12,-342.243840,0.118645,-342.117060,0.040404,0.040157,-342.157464,-342.157217,298.15,-0.24595,-0.01835,-0.13215,0.22760,0.03836,86.0359,67.1777,3.7835,956.295,276.031313,379.163951,0.917791,-0.04728,0.04261,1.41491,92.026085,95.848399,6.671932,1.833779,3.266642,no data,no data,no data


processing prefix: pyrd
Goodvibes function has completed
Frontier orbitals function has completed
Polarizability function has completed
Dipole function has completed
Volume function has completed
SASA function has completed
NBO function has completed for ['C1', 'C2']
Distance function has completed for [['C1', 'C2']]
Vbur scan function has completed for ['C1', 'C2'] from 2 to 2
Morfeus Sterimol function has completed for [['C1', 'C2']]
****No Natural Bond Order section found in: 2.log_files\pyrd1_conf-1_openshell.log
****No Natural Bond Order section found in: 2.log_files\pyrd2_conf-1_openshell.log
****No Natural Bond Order section found in: 2.log_files\pyrd3_conf-1_openshell.log
****No Natural Bond Order section found in: 2.log_files\pyrd4_conf-1_openshell_resubmit.log
****No Natural Bond Order section found in: 2.log_files\pyrd5_conf-1_openshell.log
****No Natural Bond Order section found in: 2.log_files\pyrd6_conf-1_openshell_resubmit.log
****No Natural Bond Order section found in: 

,log_name,C3,C4,N5,C6,C7,C2,C1,H1,E_spc (Hartree),ZPE(Hartree),H_spc(Hartree),T*S,T*qh_S,G(T)_spc(Hartree),qh_G(T)_spc(Hartree),T,HOMO,LUMO,μ,η,ω,polar_iso(Debye),polar_aniso(Debye),dipole(Debye),volume(Bohr_radius³/mol),SASA_surface_area(Å²),SASA_volume(Å³),SASA_sphericity,NBO_charge_C1,NBO_charge_C2,distance_C1_C2(Å),%Vbur_C1_2.0Å,%Vbur_C2_2.0Å,Sterimol_L_C1_C2(Å)_morfeus,Sterimol_B1_C1_C2(Å)_morfeus,Sterimol_B5_C1_C2(Å)_morfeus,C1_C2_Bond_Order,Natural_Valency_C1,Natural_Valency_C2
0,2.log_files\pyrd1_conf-1_openshell,C5,C4,N3,C11,C10,C2,C1,H12,-440.561375,0.150098,-440.402350,0.042178,0.042100,-440.444528,-440.444451,298.15,-0.24440,-0.02204,-0.133220,0.22236,0.03991,131.9460,128.7710,1.7939,1180.592,320.992861,463.787080,0.902681,-0.25346,0.07864,1.40454,84.765625,95.903280,8.781410,1.701110,4.632177,no data,no data,no data
1,2.log_files\pyrd2_conf-1_openshell,C11,N10,C9,C4,C3,C2,C1,H12,-440.561089,0.150066,-440.402068,0.042236,0.042144,-440.444304,-440.444212,298.15,-0.23505,-0.02558,-0.130315,0.20947,0.04054,132.4050,133.3240,2.0009,1135.998,319.412785,462.539515,0.905519,-0.28583,-0.17875,1.39820,85.059401,96.878228,8.964052,1.700586,4.320888,no data,no data,no data
2,2.log_files\pyrd3_conf-1_openshell,C3,C4,N5,C6,C7,C2,C1,H12,-440.560232,0.150251,-440.401042,0.042277,0.042162,-440.443318,-440.443204,298.15,-0.24968,-0.01785,-0.133765,0.23183,0.03859,126.3060,98.8442,2.4177,1254.516,314.502799,457.406417,0.912840,-0.26455,-0.08787,1.39449,85.295067,96.855630,7.014964,1.700657,5.719340,no data,no data,no data
3,2.log_files\pyrd4_conf-1_openshell_resubmit,C5,C4,N3,C11,C6,C2,C1,H12,-440.559301,0.150203,-440.400139,0.042404,0.042207,-440.442543,-440.442346,298.15,-0.24067,-0.01823,-0.129450,0.22244,0.03767,127.0200,98.3721,2.2854,1170.762,314.512816,457.426857,0.912838,-0.26573,0.09998,1.39438,85.117510,95.835486,6.997804,1.701246,5.730722,no data,no data,no data
4,2.log_files\pyrd5_conf-1_openshell,C3,N11,C10,C9,C4,C2,C1,H12,-440.560694,0.150126,-440.401621,0.042257,0.042156,-440.443878,-440.443777,298.15,-0.23259,-0.02679,-0.129690,0.20580,0.04086,131.9510,129.6920,2.2547,1026.284,319.472790,462.681244,0.905534,-0.27654,0.05081,1.39883,84.820506,95.906508,8.950659,1.700000,4.389249,no data,no data,no data
5,2.log_files\pyrd6_conf-1_openshell_resubmit,C3,N4,C5,C6,C11,C2,C1,H12,-440.559622,0.150164,-440.400480,0.042347,0.042242,-440.442827,-440.442721,298.15,-0.23124,-0.02320,-0.127220,0.20804,0.03890,126.2400,98.7645,2.5006,1392.394,312.932734,456.257099,0.915882,-0.30029,-0.14147,1.38993,85.340263,96.891142,7.039591,1.700735,5.713417,no data,no data,no data
6,2.log_files\pyrd7_conf-1_openshell,C10,C5,N4,C12,C11,C3,C2,H16,-479.875564,0.178346,-479.686584,0.046392,0.045993,-479.732976,-479.732577,298.15,-0.23189,-0.01792,-0.124905,0.21397,0.03646,146.8590,138.5890,1.4327,1429.909,350.142838,512.343837,0.884328,-0.04447,0.09014,1.41226,92.029313,95.822572,8.816196,1.836186,4.555269,no data,no data,no data
7,2.log_files\pyrd8_conf-1_openshell,C12,N11,C10,C5,C4,C3,C2,H16,-479.873287,0.178143,-479.684361,0.047055,0.046292,-479.731416,-479.730653,298.15,-0.22683,-0.02062,-0.123725,0.20621,0.03712,146.4100,141.7130,2.5203,1468.267,347.532628,510.647739,0.889002,-0.06647,-0.17461,1.40546,92.026085,96.823347,8.980802,1.811731,4.283140,no data,no data,no data
8,2.log_files\pyrd9_conf-2_openshell,C4,C5,N6,C7,C12,C3,C2,H16,-479.871736,0.178295,-479.682639,0.047010,0.046357,-479.729649,-479.728996,298.15,-0.24116,-0.01411,-0.127635,0.22705,0.03587,140.2120,108.8410,3.0645,1319.129,341.672637,504.196658,0.896618,-0.04751,-0.08237,1.40431,92.203642,96.858858,6.993618,1.885956,5.731829,no data,no data,no data
9,2.log_files\pyrd10_conf-2_openshell,C6,C5,N4,C12,C7,C3,C2,H16,-479.872517,0.178369,-479.683438,0.046668,0.046156,-479.730107,-479.729594,298.15,-0.22966,-0.01353,-0.121595,0.21613,0.03420,142.2250,111.1280,1.7279,1526.816,343.782804,505.395992,0.892527,-0.05043,0.10865,1.40336,92.284349,95.761235,6.976376,1.906471,5.744031,no data

processing prefix: pyrmd
Goodvibes function has completed
Frontier orbitals function has completed
Polarizability function has completed
Dipole function has completed
Volume function has completed
SASA function has completed
NBO function has completed for ['C1', 'C2']
Distance function has completed for [['C1', 'C2']]
Vbur scan function has completed for ['C1', 'C2'] from 2 to 2
Morfeus Sterimol function has completed for [['C1', 'C2']]
****No Natural Bond Order section found in: 2.log_files\pyrmd1_conf-1_openshell.log
****No Natural Bond Order section found in: 2.log_files\pyrmd2_conf-1_openshell_resubmit.log
****No Natural Bond Order section found in: 2.log_files\pyrmd3_conf-2_openshell.log
****No Natural Bond Order section found in: 2.log_files\pyrmd4_conf-2_openshell.log
****No Natural Bond Order section found in: 2.log_files\pyrmd5_conf-1_openshell.log
****No Natural Bond Order section found in: 2.log_files\pyrmd6_conf-1_openshell.log
****No Natural Bond Order section found in: 2.

,log_name,N3,C4,C5,C6,N7,C2,C1,H1,E_spc (Hartree),ZPE(Hartree),H_spc(Hartree),T*S,T*qh_S,G(T)_spc(Hartree),qh_G(T)_spc(Hartree),T,HOMO,LUMO,μ,η,ω,polar_iso(Debye),polar_aniso(Debye),dipole(Debye),volume(Bohr_radius³/mol),SASA_surface_area(Å²),SASA_volume(Å³),SASA_sphericity,NBO_charge_C1,NBO_charge_C2,distance_C1_C2(Å),%Vbur_C1_2.0Å,%Vbur_C2_2.0Å,Sterimol_L_C1_C2(Å)_morfeus,Sterimol_B1_C1_C2(Å)_morfeus,Sterimol_B5_C1_C2(Å)_morfeus,C1_C2_Bond_Order,Natural_Valency_C1,Natural_Valency_C2
0,2.log_files\pyrmd1_conf-1_openshell,N3,C4,C5,C10,N11,C2,C1,H12,-456.606650,0.138348,-456.459512,0.042030,0.041922,-456.501542,-456.501434,298.15,-0.25543,-0.03605,-0.145740,0.21938,0.04841,124.9660,120.8190,2.0433,1341.444,315.382418,452.714086,0.904057,-0.23461,0.24656,1.40558,84.649406,94.718492,8.798129,1.700000,4.412512,no data,no data,no data
1,2.log_files\pyrmd2_conf-1_openshell_resubmit,C10,N9,C8,C3,N11,C2,C1,H12,-456.605897,0.138504,-456.458573,0.042174,0.042008,-456.500747,-456.500581,298.15,-0.26818,-0.02751,-0.147845,0.24067,0.04541,120.1790,94.0428,3.1499,1127.141,309.232375,446.826313,0.914026,-0.22870,0.12255,1.40047,84.994835,95.919421,7.039504,1.700000,5.714447,no data,no data,no data
2,2.log_files\pyrmd3_conf-2_openshell,N12,C11,C10,C5,N4,C3,C2,H16,-495.921387,0.166583,-495.744300,0.046266,0.045810,-495.790566,-495.790110,298.15,-0.24140,-0.03198,-0.136690,0.20942,0.04461,140.1510,132.3330,1.3452,1262.547,344.972397,501.532464,0.884910,-0.03348,0.25925,1.41232,92.013171,94.653926,8.825994,1.837987,4.376307,no data,no data,no data
3,2.log_files\pyrmd4_conf-2_openshell,C11,N10,C9,C4,N12,C3,C2,H16,-495.920530,0.166770,-495.743211,0.046400,0.045928,-495.789611,-495.789140,298.15,-0.25474,-0.02200,-0.138370,0.23274,0.04113,135.3840,107.0520,3.2273,1398.756,338.512353,494.969402,0.893913,-0.02278,0.13383,1.40874,92.197185,95.874225,7.019439,1.905620,5.728590,no data,no data,no data
4,2.log_files\pyrmd5_conf-1_openshell,N7,C6,C5,C4,N3,C2,C1,H8,-302.968683,0.091296,-302.871036,0.035927,0.035926,-302.906962,-302.906962,298.15,-0.26667,-0.01019,-0.138430,0.25648,0.03736,71.4977,55.5619,1.5049,945.732,247.781552,330.992930,0.933887,-0.23157,0.25234,1.41069,84.555785,94.744318,6.689875,1.700000,3.254866,no data,no data,no data
5,2.log_files\pyrmd6_conf-1_openshell,C4,N3,C7,C6,N5,C2,C1,H8,-302.968958,0.091418,-302.871172,0.035954,0.035957,-302.907127,-302.907130,298.15,-0.27603,-0.00258,-0.139305,0.27345,0.03548,70.8699,51.7602,2.7823,945.261,246.491464,330.152017,0.937184,-0.23063,0.08952,1.40990,84.810821,95.935563,6.127667,1.700185,3.266709,no data,no data,no data
6,2.log_files\pyrmd7_conf-1_openshell,C5,N4,C3,C7,N6,C2,C1,H8,-302.967350,0.091131,-302.869788,0.036057,0.036061,-302.905845,-302.905849,298.15,-0.25574,-0.01669,-0.136215,0.23905,0.03881,71.6142,55.9689,2.5608,848.078,245.191403,329.294582,0.940521,-0.27768,-0.21648,1.40247,85.033574,96.868543,6.692532,1.700430,3.262758,no data,no data,no data
7,2.log_files\pyrmd8_conf-1_openshell,N8,C7,C6,C5,N4,C3,C2,H12,-342.283197,0.119588,-342.155586,0.040193,0.039967,-342.195779,-342.195552,298.15,-0.24836,-0.00639,-0.127375,0.24197,0.03353,85.7985,67.4560,0.9608,923.709,277.541528,379.967527,0.914086,-0.03525,0.26549,1.41693,92.029313,94.692665,6.699424,1.834871,3.295594,no data,no data,no data
8,2.log_files\pyrmd9_conf-1_openshell,C5,N4,C8,C7,N6,C3,C2,H12,-342.283768,0.119780,-342.155963,0.040194,0.039985,-342.196157,-342.195948,298.15,-0.25726,0.00186,-0.127700,0.25912,0.03147,85.0959,63.1108,2.9955,927.913,275.981436,378.821320,0.917404,-0.03116,0.10300,1.41634,92.013171,95.816116,6.140020,1.833860,3.270050,no data,no data,no data
9,2.log_files\pyrmd10_conf-1_openshell,C6,N5,C4,C8,N7,C3,C2,H12,-342.279133,0.119310,-342.151604,0.040946,0.040377,-342.192551,-342.191982,298.15,-0.24318,-0.01280,-0.127990,0.23038,0.03555,84.8527,64.1425,3.1796,1029.248,272.941241,377.026729,0.924690,-0.06472,-0.20856,1.41067,92.035770,96.878228,6.701352,1.809335,3.264937,no data,no data,no data


processing prefix: pyrz
Goodvibes function has completed
Frontier orbitals function has completed
Polarizability function has completed
Dipole function has completed
Volume function has completed
SASA function has completed
NBO function has completed for ['C1', 'C2']
Distance function has completed for [['C1', 'C2']]
Vbur scan function has completed for ['C1', 'C2'] from 2 to 2
Morfeus Sterimol function has completed for [['C1', 'C2']]
****No Natural Bond Order section found in: 2.log_files\pyrz1_conf-1_openshell.log
****No Natural Bond Order section found in: 2.log_files\pyrz2_conf-1_openshell.log
****No Natural Bond Order section found in: 2.log_files\pyrz3_conf-1_openshell.log
****No Natural Bond Order section found in: 2.log_files\pyrz4_conf-1_openshell.log
Natural Bond Order function has completed for [['C1', 'C2']]
****No Natural Atomic Valencies section found in: 2.log_files\pyrz1_conf-1_openshell.log
****No Natural Atomic Valencies section found in: 2.log_files\pyrz2_conf-1_ope

,log_name,C3,N4,C5,C6,N7,C2,C1,H1,E_spc (Hartree),ZPE(Hartree),H_spc(Hartree),T*S,T*qh_S,G(T)_spc(Hartree),qh_G(T)_spc(Hartree),T,HOMO,LUMO,μ,η,ω,polar_iso(Debye),polar_aniso(Debye),dipole(Debye),volume(Bohr_radius³/mol),SASA_surface_area(Å²),SASA_volume(Å³),SASA_sphericity,NBO_charge_C1,NBO_charge_C2,distance_C1_C2(Å),%Vbur_C1_2.0Å,%Vbur_C2_2.0Å,Sterimol_L_C1_C2(Å)_morfeus,Sterimol_B1_C1_C2(Å)_morfeus,Sterimol_B5_C1_C2(Å)_morfeus,C1_C2_Bond_Order,Natural_Valency_C1,Natural_Valency_C2
0,2.log_files\pyrz1_conf-1_openshell,C11,N10,C5,C4,N3,C2,C1,H12,-456.602010,0.138133,-456.455080,0.042010,0.041907,-456.497090,-456.496987,298.15,-0.25413,-0.03682,-0.145475,0.21731,0.04869,127.9930,131.2830,0.3188,1389.023,315.092407,452.482255,0.904581,-0.25207,0.03205,1.39906,84.804365,95.877454,8.806875,1.700102,4.366746,no data,no data,no data
1,2.log_files\pyrz2_conf-1_openshell,C12,N11,C10,C5,N4,C3,C2,H16,-495.916763,0.166412,-495.739849,0.046220,0.045792,-495.786069,-495.785641,298.15,-0.24152,-0.03164,-0.136580,0.20988,0.04444,143.2130,142.6610,1.1798,1441.282,344.962385,501.577355,0.884989,-0.04118,0.04065,1.40527,92.129390,95.861312,8.831281,1.836373,4.332991,no data,no data,no data
2,2.log_files\pyrz3_conf-1_openshell,C7,N6,C5,C4,N3,C2,C1,H8,-302.962267,0.091034,-302.864850,0.035982,0.035983,-302.900832,-302.900833,298.15,-0.25894,-0.01748,-0.138210,0.24146,0.03956,73.1261,59.8800,0.5359,790.755,246.651475,330.391736,0.937029,-0.25894,0.02576,1.40272,84.881844,95.841942,6.685710,1.700000,3.270578,no data,no data,no data
3,2.log_files\pyrz4_conf-1_openshell,C8,N7,C6,C5,N4,C3,C2,H12,-342.274140,0.119119,-342.146813,0.041154,0.040404,-342.187966,-342.187216,298.15,-0.24589,-0.01522,-0.130555,0.23067,0.03695,86.4426,69.3885,1.1923,875.051,274.331312,378.015692,0.921613,-0.05304,0.03713,1.41168,91.964747,95.896823,6.697989,1.808463,3.266766,no data,no data,no data


## 3.1 Save collected properties to Excel and pickle file

In [5]:
for prefix, df in atom_map_df_all.items():
    # save the pandas dataframe to a xlsx file
    with pd.ExcelWriter(temp_folder + os.sep + prefix + "_extracted_properties.xlsx") as writer:
        df.to_excel(writer)

# **4. Post-processing**

In [6]:
import re
import pandas as pd
import numpy as np
from tabulate import tabulate

In [7]:
# for numerically named compounds, prefix is any text common to all BEFORE the number and suffix is common to all AFTER the number
# this is a template for our files that are all named "AcXXX_clust-X.log" or "AcXXX_conf-X.log"
suffix = "_"
prefixs = {}
for file in glob.glob("*.xlsx", root_dir=atom_mappings_folder):
    key = re.search(r"^(\D+)_atom_map", file)
    if key and key.group(1) in prefixs:
        prefixs[key.group(1)].append(file)
    else:
        prefixs[key.group(1)] = [file]

# columns that provide atom mapping information are dropped, not need if these columns contain cells that cannot be convert to float
# e.g. atom_columns_to_drop = ["C3", "C4", "C5", "N1", "C1", "C2"]
atom_columns_to_drop = []

# title of the column for the energy you want to use for boltzmann averaging and lowest E conformer determination
energy_col_header = "G(T)_spc(Hartree)"

### Option to import an Excel sheet if you're using properties or energies collected outside of this notebook

##### If you would like to use post-processing functionality (i.e. Boltzmann averaging, lowest E conformers, etc.) you can read in a dataframe with properties (e.g. QikProp properties) or energies (e.g. if you don't/can't run linked jobs) collected outside of this notebook. 

In [8]:
atom_map_df_all = {}

for prefix in prefixs:
    df = pd.read_excel(
        temp_folder + os.sep +
        prefix + "_extracted_properties.xlsx",
        "Sheet1",
        index_col=0,
        header=0,
        engine="openpyxl",
    )
    display(df.head(2))
    atom_map_df_all[prefix] = df.copy(deep=True)

,log_name,N3,N4,C5,C6,C7,C2,C1,H1,E_spc (Hartree),ZPE(Hartree),H_spc(Hartree),T*S,T*qh_S,G(T)_spc(Hartree),qh_G(T)_spc(Hartree),T,HOMO,LUMO,μ,η,ω,polar_iso(Debye),polar_aniso(Debye),dipole(Debye),volume(Bohr_radius³/mol),SASA_surface_area(Å²),SASA_volume(Å³),SASA_sphericity,NBO_charge_C1,NBO_charge_C2,distance_C1_C2(Å),%Vbur_C1_2.0Å,%Vbur_C2_2.0Å,Sterimol_L_C1_C2(Å)_morfeus,Sterimol_B1_C1_C2(Å)_morfeus,Sterimol_B5_C1_C2(Å)_morfeus,C1_C2_Bond_Order,Natural_Valency_C1,Natural_Valency_C2
0,2.log_files\pyrdz1_conf-1_openshell,N7,N6,C5,C4,C3,C2,C1,H8,-302.930027,0.090330,-302.833243,0.036086,0.036089,-302.869329,-302.869332,298.15,-0.26296,-0.02244,-0.14270,0.24052,0.04233,71.9025,55.6433,4.0685,794.393,246.781337,330.850635,0.937403,-0.25250,0.02928,1.40721,84.743027,95.896823,6.652549,1.700000,3.261975,no data,no data,no data
1,2.log_files\pyrdz2_conf-1_openshell,C3,N4,N5,C6,C7,C2,C1,H8,-302.930293,0.090405,-302.833411,0.036145,0.036148,-302.869556,-302.869559,298.15,-0.27376,-0.01060,-0.14218,0.26316,0.03841,71.7614,53.4032,4.6724,1032.521,245.501238,329.913006,0.940510,-0.24648,-0.15464,1.40277,84.894757,96.923425,6.157663,1.700202,3.270340,no data,no data,no data


,log_name,C3,C4,N5,C6,C7,C2,C1,H1,E_spc (Hartree),ZPE(Hartree),H_spc(Hartree),T*S,T*qh_S,G(T)_spc(Hartree),qh_G(T)_spc(Hartree),T,HOMO,LUMO,μ,η,ω,polar_iso(Debye),polar_aniso(Debye),dipole(Debye),volume(Bohr_radius³/mol),SASA_surface_area(Å²),SASA_volume(Å³),SASA_sphericity,NBO_charge_C1,NBO_charge_C2,distance_C1_C2(Å),%Vbur_C1_2.0Å,%Vbur_C2_2.0Å,Sterimol_L_C1_C2(Å)_morfeus,Sterimol_B1_C1_C2(Å)_morfeus,Sterimol_B5_C1_C2(Å)_morfeus,C1_C2_Bond_Order,Natural_Valency_C1,Natural_Valency_C2
0,2.log_files\pyrd1_conf-1_openshell,C5,C4,N3,C11,C10,C2,C1,H12,-440.561375,0.150098,-440.402350,0.042178,0.042100,-440.444528,-440.444451,298.15,-0.24440,-0.02204,-0.133220,0.22236,0.03991,131.946,128.771,1.7939,1180.592,320.992861,463.787080,0.902681,-0.25346,0.07864,1.40454,84.765625,95.903280,8.781410,1.701110,4.632177,no data,no data,no data
1,2.log_files\pyrd2_conf-1_openshell,C11,N10,C9,C4,C3,C2,C1,H12,-440.561089,0.150066,-440.402068,0.042236,0.042144,-440.444304,-440.444212,298.15,-0.23505,-0.02558,-0.130315,0.20947,0.04054,132.405,133.324,2.0009,1135.998,319.412785,462.539515,0.905519,-0.28583,-0.17875,1.39820,85.059401,96.878228,8.964052,1.700586,4.320888,no data,no data,no data


,log_name,N3,C4,C5,C6,N7,C2,C1,H1,E_spc (Hartree),ZPE(Hartree),H_spc(Hartree),T*S,T*qh_S,G(T)_spc(Hartree),qh_G(T)_spc(Hartree),T,HOMO,LUMO,μ,η,ω,polar_iso(Debye),polar_aniso(Debye),dipole(Debye),volume(Bohr_radius³/mol),SASA_surface_area(Å²),SASA_volume(Å³),SASA_sphericity,NBO_charge_C1,NBO_charge_C2,distance_C1_C2(Å),%Vbur_C1_2.0Å,%Vbur_C2_2.0Å,Sterimol_L_C1_C2(Å)_morfeus,Sterimol_B1_C1_C2(Å)_morfeus,Sterimol_B5_C1_C2(Å)_morfeus,C1_C2_Bond_Order,Natural_Valency_C1,Natural_Valency_C2
0,2.log_files\pyrmd1_conf-1_openshell,N3,C4,C5,C10,N11,C2,C1,H12,-456.606650,0.138348,-456.459512,0.042030,0.041922,-456.501542,-456.501434,298.15,-0.25543,-0.03605,-0.145740,0.21938,0.04841,124.966,120.8190,2.0433,1341.444,315.382418,452.714086,0.904057,-0.23461,0.24656,1.40558,84.649406,94.718492,8.798129,1.7,4.412512,no data,no data,no data
1,2.log_files\pyrmd2_conf-1_openshell_resubmit,C10,N9,C8,C3,N11,C2,C1,H12,-456.605897,0.138504,-456.458573,0.042174,0.042008,-456.500747,-456.500581,298.15,-0.26818,-0.02751,-0.147845,0.24067,0.04541,120.179,94.0428,3.1499,1127.141,309.232375,446.826313,0.914026,-0.22870,0.12255,1.40047,84.994835,95.919421,7.039504,1.7,5.714447,no data,no data,no data


,log_name,C3,N4,C5,C6,N7,C2,C1,H1,E_spc (Hartree),ZPE(Hartree),H_spc(Hartree),T*S,T*qh_S,G(T)_spc(Hartree),qh_G(T)_spc(Hartree),T,HOMO,LUMO,μ,η,ω,polar_iso(Debye),polar_aniso(Debye),dipole(Debye),volume(Bohr_radius³/mol),SASA_surface_area(Å²),SASA_volume(Å³),SASA_sphericity,NBO_charge_C1,NBO_charge_C2,distance_C1_C2(Å),%Vbur_C1_2.0Å,%Vbur_C2_2.0Å,Sterimol_L_C1_C2(Å)_morfeus,Sterimol_B1_C1_C2(Å)_morfeus,Sterimol_B5_C1_C2(Å)_morfeus,C1_C2_Bond_Order,Natural_Valency_C1,Natural_Valency_C2
0,2.log_files\pyrz1_conf-1_openshell,C11,N10,C5,C4,N3,C2,C1,H12,-456.602010,0.138133,-456.455080,0.04201,0.041907,-456.497090,-456.496987,298.15,-0.25413,-0.03682,-0.145475,0.21731,0.04869,127.993,131.283,0.3188,1389.023,315.092407,452.482255,0.904581,-0.25207,0.03205,1.39906,84.804365,95.877454,8.806875,1.700102,4.366746,no data,no data,no data
1,2.log_files\pyrz2_conf-1_openshell,C12,N11,C10,C5,N4,C3,C2,H16,-495.916763,0.166412,-495.739849,0.04622,0.045792,-495.786069,-495.785641,298.15,-0.24152,-0.03164,-0.136580,0.20988,0.04444,143.213,142.661,1.1798,1441.282,344.962385,501.577355,0.884989,-0.04118,0.04065,1.40527,92.129390,95.861312,8.831281,1.836373,4.332991,no data,no data,no data


## 4.1 Generating a list of compounds that have conformational ensembles

**ONLY RUN THE AUTOMATED OR THE MANUAL CELL, NOT BOTH**

**AUTOMATED:** if your compounds are named consistenly, this section generates your compound list based on the similar naming structure

In [9]:
compound_list_all = {}

for prefix, df in atom_map_df_all.items():
    print(f"processing prefix: {prefix}")
    compound_list = []

    for index, row in df.iterrows():
        log_file = row["log_name"]  # read file name from df
        # first split by "\" take the last part
        log_file = log_file.split(os.sep)[-1]
        prefix_and_compound = log_file.split(str(suffix))
        compound = prefix_and_compound[0].split(str(prefix))  # splits again to get "XXX" (entry 1) (and we don't use the empty string "" (entry 0))
        compound_list.append(compound[1])

    compound_list = list(set(compound_list))  # removes duplicate stuctures that result from having conformers of each
    compound_list.sort(key=lambda x: int(re.search(r"\d+", x).group()))  # reorders numerically (not sure if it reorders alphabetically)
    print(f"items numbering: {compound_list}")
    compound_list_all[prefix] = compound_list

    # this should generate a list that looks like this: ['24', '27', '34', '48']

processing prefix: pyrdz
items numbering: ['1', '2', '3']
processing prefix: pyrd
items numbering: ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18']
processing prefix: pyrmd
items numbering: ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10']
processing prefix: pyrz
items numbering: ['1', '2', '3', '4']


## 4.2 Post-processing to get properties for each compound

##### changes made in 8/30/2024 <br> 1. avoid divide by zero error in the Boltzmann averaging, the original code had the if block order reversed, which caused the error. <br> 2. data cleaning by remove columns contain cell that cannot be converted to float. <br> 3. concat all data into row before concat them into the final dataframe. The originl modify individual cells which result in fragmented and raise performance warning.

In [10]:
all_df_master_for_all_prefixes = {}
properties_df_master_for_all_prefixes = {}

for prefix, df in atom_map_df_all.items():
    compound_list = compound_list_all[prefix]
    all_df_master = pd.DataFrame(columns=[])
    properties_df_master = pd.DataFrame(columns=[])

    for compound in compound_list:
        # defines the common start to all files using the input above
        substring = log_files_folder + os.sep + str(prefix) + str(compound) + str(suffix)

        # makes a data frame for one compound at a time for post-processing
        valuesdf = df[df["log_name"].str.startswith(substring)]
        valuesdf = valuesdf.drop(columns=atom_columns_to_drop)
        valuesdf = valuesdf.reset_index(
            drop=True
        )  # you must re-index otherwise the 2nd, 3rd, etc. compounds fail

        # filter column that are characters, we will attempt to convert them to numeric numbers, if fail, we will drop them
        for column in valuesdf:
            try:
                # exclude column "log_name"
                if column == "log_name":
                    continue
                valuesdf[column] = pd.to_numeric(valuesdf[column])
            except:
                print(f"Column {column} contains non-numeric values")
                valuesdf = valuesdf.drop(columns=column)
                valuesdf = valuesdf.reset_index(
                    drop=True
                )  # reset the index after dropping columns

        # define columns that won't be included in summary properties or are treated differently because they don't make sense to Boltzmann average
        non_boltz_columns = [
            "G(Hartree)",
            "∆G(Hartree)",
            "∆G(kcal/mol)",
            "e^(-∆G/RT)",
            "Mole Fraction",
        ]  # don't boltzman average columns containing these strings in the column label
        reg_avg_columns = [
            "CPU_time_total(hours)",
            "Wall_time_total(hours)",
        ]  # don't boltzmann average these either, we average them in case that is helpful
        gv_extra_columns = [
            "G(T)_spc(Hartree)",
        ]
        gv_extra_columns.remove(str(energy_col_header))

        # calculate the summary properties based on all conformers (Boltzmann Average, Minimum, Maximum, Boltzmann Weighted Std)
        valuesdf["∆G(Hartree)"] = (
            valuesdf[energy_col_header] - valuesdf[energy_col_header].min()
        )
        valuesdf["∆G(kcal/mol)"] = valuesdf["∆G(Hartree)"] * 627.5
        valuesdf["e^(-∆G/RT)"] = np.exp(
            (valuesdf["∆G(kcal/mol)"] * -1000) / (1.987204 * 298.15)
        )  # R is in cal/(K*mol)
        valuesdf["Mole Fraction"] = valuesdf["e^(-∆G/RT)"] / valuesdf["e^(-∆G/RT)"].sum()
        values_boltz_row = []
        values_min_row = []
        values_max_row = []
        values_boltz_stdev_row = []
        values_range_row = []
        values_exclude_columns = []

        for column in valuesdf:
            if "log_name" in column:
                values_boltz_row.append("Boltzmann Averages")
                values_min_row.append("Ensemble Minimum")
                values_max_row.append("Ensemble Maximum")
                values_boltz_stdev_row.append("Boltzmann Standard Deviation")
                values_range_row.append("Ensemble Range")
                values_exclude_columns.append(column)  # used later to build final dataframe
            elif any(phrase in column for phrase in non_boltz_columns) or any(
                phrase in column for phrase in gv_extra_columns
            ):
                values_boltz_row.append("")
                values_min_row.append("")
                values_max_row.append("")
                values_boltz_stdev_row.append("")
                values_range_row.append("")
            elif any(phrase in column for phrase in reg_avg_columns):
                values_boltz_row.append(
                    valuesdf[column].mean()
                )  # intended to print the average CPU/wall time in the boltz column
                values_min_row.append("")
                values_max_row.append("")
                values_boltz_stdev_row.append("")
                values_range_row.append("")
            else:
                valuesdf[column] = pd.to_numeric(
                    valuesdf[column]
                )  # to hopefully solve the error that sometimes occurs where the float(Mole Fraction) cannot be mulitplied by the string(property)
                values_boltz_row.append(
                    (valuesdf[column] * valuesdf["Mole Fraction"]).sum()
                )
                values_min_row.append(valuesdf[column].min())
                values_max_row.append(valuesdf[column].max())
                values_range_row.append(valuesdf[column].max() - valuesdf[column].min())

                # this section generates the weighted std deviation (weighted by mole fraction)
                # formula: https://www.statology.org/weighted-standard-deviation-excel/

                boltz = (valuesdf[column] * valuesdf["Mole Fraction"]).sum()  # number
                delta_values_sq = []

                # makes a list of the "deviation" for each conformer
                for index, row in valuesdf.iterrows():
                    value = row[column]
                    delta_value_sq = (value - boltz) ** 2
                    delta_values_sq.append(delta_value_sq)

                # w is list of weights (i.e. mole fractions)
                w = list(valuesdf["Mole Fraction"])
                # !swap the order here to avoid division by zero error
                if (
                    len(w) == 1
                ):  # if there is only one conformer in the ensemble, set the weighted standard deviation to 0
                    wstdev = 0
                # np.average(delta_values_sq, weights=w) generates sum of each (delta_value_sq * mole fraction)
                else:
                    wstdev = np.sqrt(
                        (np.average(delta_values_sq, weights=w))
                        / (((len(w) - 1) / len(w)) * np.sum(w))
                    )
                values_boltz_stdev_row.append(wstdev)

        valuesdf.loc[len(valuesdf)] = values_boltz_row
        valuesdf.loc[len(valuesdf)] = values_boltz_stdev_row
        valuesdf.loc[len(valuesdf)] = values_min_row
        valuesdf.loc[len(valuesdf)] = values_max_row
        valuesdf.loc[len(valuesdf)] = values_range_row

        # final output format is built here:
        explicit_order_front_columns = [
            "log_name",
            energy_col_header,
            "∆G(Hartree)",
            "∆G(kcal/mol)",
            "e^(-∆G/RT)",
            "Mole Fraction",
        ]

        # reorders the dataframe using front columns defined above
        valuesdf = valuesdf[
            explicit_order_front_columns
            + [
                col
                for col in valuesdf.columns
                if col not in explicit_order_front_columns
                and col not in values_exclude_columns
            ]
        ]

        # determine the index of the lowest energy conformer
        low_e_index = valuesdf[valuesdf["∆G(Hartree)"] == 0].index.tolist()
        # copy the row to a new_row with the name of the log changed to Lowest E Conformer
        new_row = pd.DataFrame(valuesdf.loc[low_e_index[0]]).T
        new_row["log_name"] = "Lowest E Conformer"

        valuesdf = pd.concat([valuesdf, new_row], ignore_index=True, axis=0)

        # ------------------------------EDIT THIS SECTION IF YOU WANT A SPECIFIC CONFORMER----------------------------------
        # if you want all properties for a conformer with a particular property (i.e. all properties for the Vbur_min conformer)
        # this template can be adjusted for min/max/etc.

        # find the index for the min or max column:
        ensemble_min_index = valuesdf[
            valuesdf["log_name"] == "Ensemble Minimum"
        ].index.tolist()

        # find the min or max value of the property (based on index above)
        # saves the value in a list (min_value) with one entry (this is why we call min_value[0])
        min_value = valuesdf.loc[ensemble_min_index, "%Vbur_C1_2.0Å"].tolist()
        vbur_min_index = valuesdf[valuesdf["%Vbur_C1_2.0Å"] == min_value[0]].index.tolist()

        # copy the row to a new_row with the name of the log changed to Property_min_conformer
        new_row = pd.DataFrame(valuesdf.loc[vbur_min_index[0]]).T
        new_row["log_name"] = "%Vbur_C1_2.0Å_min_Conformer"

        valuesdf = pd.concat([valuesdf, new_row], ignore_index=True, axis=0)

        # --------------------------------------------------------------------------------------------------------------------

        # !here we define a list of properties we only want the minimal value for
        min_property_list = [
            "E_spc (Hartree)",
            "H_spc(Hartree)",
            "T",
            "T*S",
            "T*qh_S",
            "ZPE(Hartree)",
            "qh_G(T)_spc(Hartree)",
            "G(T)_spc(Hartree)",
        ]
        # extract the "Lowest E Conformer" row out of the dataframe
        Low_E_Conformer_row = pd.DataFrame(
            valuesdf.loc[valuesdf["log_name"] == "Lowest E Conformer"]
        )
        # extract the "Boltzmann Averages" row out of the dataframe
        Boltz_Avg_row = pd.DataFrame(
            valuesdf.loc[valuesdf["log_name"] == "Boltzmann Averages"]
        )
        # display(valuesdf) # debug display for finding the row index
        # display(Low_E_Conformer_row)

        # appends the frame to the master output
        all_df_master = pd.concat([all_df_master, valuesdf])

        # drop all the individual conformers
        dropindex = valuesdf[valuesdf["log_name"].str.startswith(substring)].index
        valuesdf = valuesdf.drop(dropindex)
        valuesdf = valuesdf.reset_index(drop=True)

        # drop the columns created to determine the mole fraction and some that
        valuesdf = valuesdf.drop(columns=explicit_order_front_columns)
        try:
            valuesdf = valuesdf.drop(columns=gv_extra_columns)
        except:
            pass
        try:
            valuesdf = valuesdf.drop(columns=reg_avg_columns)
        except:
            pass

        # ---------------------THIS MAY NEED TO CHANGE DEPENDING ON HOW YOU LABEL YOUR COMPOUNDS------------------------------
        compound_name = prefix + str(compound)
        # --------------------------------------------------------------------------------------------------------------------

        properties_df = pd.DataFrame({"Compound_Name": [compound_name]})

        # builds a dataframe (for each compound) by adding summary properties as new columns
        for column in valuesdf:
            # print(column)
            # the indexes need to match the values dataframe - display it to double check if you need to make changes
            # (uncomment the display(valuesdf) in row 124 of this cell)

            # create a list of headers for the properties_df
            # if you're collecting properties for a specific conformer, edit the header to reflect that, it should match the order in the valuesdf log_name column
            if column in min_property_list:
                # ! if we are working with a property that we only want the minimum value for, we only need one header
                headers = [
                    f"{column}",
                ]
                # use data from the Low_E_Conformer_row
                row_dataframe = pd.DataFrame(
                    [Low_E_Conformer_row[column].values], columns=headers
                )
            else:
                headers = [
                    f"{column}_Boltz",
                ]
                row_dataframe = pd.DataFrame([Boltz_Avg_row[column].values], columns=headers)
            # Extract values for the current column from valuesdf and create a DataFrame
            # Display the DataFrame for verification
            # display(row_dataframe)
            # Concatenate the new DataFrame to the properties_df along the columns (axis=1)
            properties_df = pd.concat([properties_df, row_dataframe], axis=1)

        # concatenates the individual acid properties df into the master properties df
        properties_df_master = pd.concat([properties_df_master, properties_df], axis=0)

    # Reset the index of the master DataFrames
    all_df_master = all_df_master.reset_index(drop=True)
    all_df_master_for_all_prefixes[prefix] = all_df_master.copy(deep=True)
    properties_df_master = properties_df_master.reset_index(drop=True)
    properties_df_master_for_all_prefixes[prefix] = properties_df_master.copy(deep=True)
    
    # Print in tabulated format
    print(tabulate(properties_df_master_for_all_prefixes[prefix], headers="keys", tablefmt="pretty"))
    print(tabulate(all_df_master_for_all_prefixes[prefix], headers="keys", tablefmt="pretty"))

Column N3 contains non-numeric values
Column N4 contains non-numeric values
Column C5 contains non-numeric values
Column C6 contains non-numeric values
Column C7 contains non-numeric values
Column C2 contains non-numeric values
Column C1 contains non-numeric values
Column H1 contains non-numeric values
Column C1_C2_Bond_Order contains non-numeric values
Column Natural_Valency_C1 contains non-numeric values
Column Natural_Valency_C2 contains non-numeric values
Column N3 contains non-numeric values
Column N4 contains non-numeric values
Column C5 contains non-numeric values
Column C6 contains non-numeric values
Column C7 contains non-numeric values
Column C2 contains non-numeric values
Column C1 contains non-numeric values
Column H1 contains non-numeric values
Column C1_C2_Bond_Order contains non-numeric values
Column Natural_Valency_C1 contains non-numeric values
Column Natural_Valency_C2 contains non-numeric values
Column N3 contains non-numeric values
Column N4 contains non-numeric val

In [11]:
# merge all the properties_df_master_for_all_prefixes into a single dataframe, combine column with the same name
properties_df_master_for_all_prefixes_merged = pd.DataFrame(columns=[])
for prefix, df in properties_df_master_for_all_prefixes.items():
    display(df.head(1))
    properties_df_master_for_all_prefixes_merged = pd.concat([properties_df_master_for_all_prefixes_merged, df], axis=0)
    properties_df_master_for_all_prefixes_merged.reset_index(drop=True, inplace=True)

print(f"Combine summary properties for all prefixes: ")
display(properties_df_master_for_all_prefixes_merged.head(5))

,Compound_Name,E_spc (Hartree),ZPE(Hartree),H_spc(Hartree),T*S,T*qh_S,qh_G(T)_spc(Hartree),T,HOMO_Boltz,LUMO_Boltz,μ_Boltz,η_Boltz,ω_Boltz,polar_iso(Debye)_Boltz,polar_aniso(Debye)_Boltz,dipole(Debye)_Boltz,volume(Bohr_radius³/mol)_Boltz,SASA_surface_area(Å²)_Boltz,SASA_volume(Å³)_Boltz,SASA_sphericity_Boltz,NBO_charge_C1_Boltz,NBO_charge_C2_Boltz,distance_C1_C2(Å)_Boltz,%Vbur_C1_2.0Å_Boltz,%Vbur_C2_2.0Å_Boltz,Sterimol_L_C1_C2(Å)_morfeus_Boltz,Sterimol_B1_C1_C2(Å)_morfeus_Boltz,Sterimol_B5_C1_C2(Å)_morfeus_Boltz
0,pyrdz1,-302.930027,0.09033,-302.833243,0.036086,0.036089,-302.869332,298.15,-0.26296,-0.02244,-0.1427,0.24052,0.04233,71.9025,55.6433,4.0685,794.393,246.781337,330.850635,0.937403,-0.2525,0.02928,1.40721,84.743027,95.896823,6.652549,1.7,3.261975


,Compound_Name,E_spc (Hartree),ZPE(Hartree),H_spc(Hartree),T*S,T*qh_S,qh_G(T)_spc(Hartree),T,HOMO_Boltz,LUMO_Boltz,μ_Boltz,η_Boltz,ω_Boltz,polar_iso(Debye)_Boltz,polar_aniso(Debye)_Boltz,dipole(Debye)_Boltz,volume(Bohr_radius³/mol)_Boltz,SASA_surface_area(Å²)_Boltz,SASA_volume(Å³)_Boltz,SASA_sphericity_Boltz,NBO_charge_C1_Boltz,NBO_charge_C2_Boltz,distance_C1_C2(Å)_Boltz,%Vbur_C1_2.0Å_Boltz,%Vbur_C2_2.0Å_Boltz,Sterimol_L_C1_C2(Å)_morfeus_Boltz,Sterimol_B1_C1_C2(Å)_morfeus_Boltz,Sterimol_B5_C1_C2(Å)_morfeus_Boltz
0,pyrd1,-440.561375,0.150098,-440.40235,0.042178,0.0421,-440.444451,298.15,-0.2444,-0.02204,-0.13322,0.22236,0.03991,131.946,128.771,1.7939,1180.592,320.992861,463.78708,0.902681,-0.25346,0.07864,1.40454,84.765625,95.90328,8.78141,1.70111,4.632177


,Compound_Name,E_spc (Hartree),ZPE(Hartree),H_spc(Hartree),T*S,T*qh_S,qh_G(T)_spc(Hartree),T,HOMO_Boltz,LUMO_Boltz,μ_Boltz,η_Boltz,ω_Boltz,polar_iso(Debye)_Boltz,polar_aniso(Debye)_Boltz,dipole(Debye)_Boltz,volume(Bohr_radius³/mol)_Boltz,SASA_surface_area(Å²)_Boltz,SASA_volume(Å³)_Boltz,SASA_sphericity_Boltz,NBO_charge_C1_Boltz,NBO_charge_C2_Boltz,distance_C1_C2(Å)_Boltz,%Vbur_C1_2.0Å_Boltz,%Vbur_C2_2.0Å_Boltz,Sterimol_L_C1_C2(Å)_morfeus_Boltz,Sterimol_B1_C1_C2(Å)_morfeus_Boltz,Sterimol_B5_C1_C2(Å)_morfeus_Boltz
0,pyrmd1,-456.60665,0.138348,-456.459512,0.04203,0.041922,-456.501434,298.15,-0.25543,-0.03605,-0.14574,0.21938,0.04841,124.966,120.819,2.0433,1341.444,315.382418,452.714086,0.904057,-0.23461,0.24656,1.40558,84.649406,94.718492,8.798129,1.7,4.412512


,Compound_Name,E_spc (Hartree),ZPE(Hartree),H_spc(Hartree),T*S,T*qh_S,qh_G(T)_spc(Hartree),T,HOMO_Boltz,LUMO_Boltz,μ_Boltz,η_Boltz,ω_Boltz,polar_iso(Debye)_Boltz,polar_aniso(Debye)_Boltz,dipole(Debye)_Boltz,volume(Bohr_radius³/mol)_Boltz,SASA_surface_area(Å²)_Boltz,SASA_volume(Å³)_Boltz,SASA_sphericity_Boltz,NBO_charge_C1_Boltz,NBO_charge_C2_Boltz,distance_C1_C2(Å)_Boltz,%Vbur_C1_2.0Å_Boltz,%Vbur_C2_2.0Å_Boltz,Sterimol_L_C1_C2(Å)_morfeus_Boltz,Sterimol_B1_C1_C2(Å)_morfeus_Boltz,Sterimol_B5_C1_C2(Å)_morfeus_Boltz
0,pyrz1,-456.60201,0.138133,-456.45508,0.04201,0.041907,-456.496987,298.15,-0.25413,-0.03682,-0.145475,0.21731,0.04869,127.993,131.283,0.3188,1389.023,315.092407,452.482255,0.904581,-0.25207,0.03205,1.39906,84.804365,95.877454,8.806875,1.700102,4.366746


Combine summary properties for all prefixes: 


,Compound_Name,E_spc (Hartree),ZPE(Hartree),H_spc(Hartree),T*S,T*qh_S,qh_G(T)_spc(Hartree),T,HOMO_Boltz,LUMO_Boltz,μ_Boltz,η_Boltz,ω_Boltz,polar_iso(Debye)_Boltz,polar_aniso(Debye)_Boltz,dipole(Debye)_Boltz,volume(Bohr_radius³/mol)_Boltz,SASA_surface_area(Å²)_Boltz,SASA_volume(Å³)_Boltz,SASA_sphericity_Boltz,NBO_charge_C1_Boltz,NBO_charge_C2_Boltz,distance_C1_C2(Å)_Boltz,%Vbur_C1_2.0Å_Boltz,%Vbur_C2_2.0Å_Boltz,Sterimol_L_C1_C2(Å)_morfeus_Boltz,Sterimol_B1_C1_C2(Å)_morfeus_Boltz,Sterimol_B5_C1_C2(Å)_morfeus_Boltz
0,pyrdz1,-302.930027,0.090330,-302.833243,0.036086,0.036089,-302.869332,298.15,-0.26296,-0.02244,-0.142700,0.24052,0.04233,71.9025,55.6433,4.0685,794.393,246.781337,330.850635,0.937403,-0.25250,0.02928,1.40721,84.743027,95.896823,6.652549,1.700000,3.261975
1,pyrdz2,-302.930293,0.090405,-302.833411,0.036145,0.036148,-302.869559,298.15,-0.27376,-0.01060,-0.142180,0.26316,0.03841,71.7614,53.4032,4.6724,1032.521,245.501238,329.913006,0.940510,-0.24648,-0.15464,1.40277,84.894757,96.923425,6.157663,1.700202,3.270340
2,pyrdz3,-342.243840,0.118645,-342.117060,0.040404,0.040157,-342.157217,298.15,-0.24595,-0.01835,-0.132150,0.22760,0.03836,86.0359,67.1777,3.7835,956.295,276.031313,379.163951,0.917791,-0.04728,0.04261,1.41491,92.026085,95.848399,6.671932,1.833779,3.266642
3,pyrd1,-440.561375,0.150098,-440.402350,0.042178,0.042100,-440.444451,298.15,-0.24440,-0.02204,-0.133220,0.22236,0.03991,131.9460,128.7710,1.7939,1180.592,320.992861,463.787080,0.902681,-0.25346,0.07864,1.40454,84.765625,95.903280,8.781410,1.701110,4.632177
4,pyrd2,-440.561089,0.150066,-440.402068,0.042236,0.042144,-440.444212,298.15,-0.23505,-0.02558,-0.130315,0.20947,0.04054,132.4050,133.3240,2.0009,1135.998,319.412785,462.539515,0.905519,-0.28583,-0.17875,1.39820,85.059401,96.878228,8.964052,1.700586,4.320888


# 5. Export the data

In [12]:
# Define the filename for the Excel file
with pd.ExcelWriter(output_folder + os.sep + "Properties_postprocessed_all_prefixes.xlsx", engine="xlsxwriter") as writer:
    for prefix, properties_df_master in properties_df_master_for_all_prefixes.items():
        print(f"Writing to Excel file for prefix: {prefix}")
        all_df_master = all_df_master_for_all_prefixes[prefix]

        all_df_master.to_excel(writer, sheet_name="All_Conformer_Properties_" + prefix, index=False)
        # automatically adjusts the width of the columns
        for column in all_df_master.columns:
            column_width = max(all_df_master[column].astype(str).map(len).max(), len(column))
            col_idx = all_df_master.columns.get_loc(column)
            writer.sheets["All_Conformer_Properties_" + prefix].set_column(col_idx, col_idx, column_width)
        properties_df_master.to_excel(writer, sheet_name="Summary_Properties_" + prefix, index=False)
        # automatically adjusts the width of the columns
        for column in properties_df_master.columns:
            column_width = max(properties_df_master[column].astype(str).map(len).max(), len(column))
            col_idx = properties_df_master.columns.get_loc(column)
            writer.sheets["Summary_Properties_" + prefix].set_column(col_idx, col_idx, column_width)

Writing to Excel file for prefix: pyrdz
Writing to Excel file for prefix: pyrd
Writing to Excel file for prefix: pyrmd
Writing to Excel file for prefix: pyrz


In [13]:
# write the combined properties_df_master_for_all_prefixes to an Excel file
with pd.ExcelWriter(output_folder + os.sep + "Summary_Properties_all.xlsx", engine="xlsxwriter") as writer:
    properties_df_master_for_all_prefixes_merged.to_excel(writer, sheet_name="Summary_Properties", index=False)
    # automatically adjusts the width of the columns
    for column in properties_df_master_for_all_prefixes_merged.columns:
        column_width = max(properties_df_master_for_all_prefixes_merged[column].astype(str).map(len).max(), len(column))
        col_idx = properties_df_master_for_all_prefixes_merged.columns.get_loc(column)
        writer.sheets["Summary_Properties"].set_column(col_idx, col_idx, column_width)